In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 15.6MB/s 
     |████████████████████████████████| 890kB 55.8MB/s 
     |████████████████████████████████| 2.9MB 52.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4db9401408bc18525b84c61f88bdc680fc7611e443eed9e3e143d80be0725b96
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 15.1MB/s 
     |████████████████████████████████| 245kB 31.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=865ed57d6cd87bf1851931866c8f3a1bdc1f797d913c2a325b2073043b89f643
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer,BertTokenizer,BertForSequenceClassification
import os
import numpy as np
import re
import tqdm
import unicodedata
import contractions
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
MAX_LEN = 512
VALID_BATCH_SIZE = 16
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
test = pd.read_csv('/content/gdrive/My Drive/lily/test.csv')
output_map = {'Animal Diseases': 0,
 'Bacterial Infections and Mycoses': 3,
 'Cardiovascular Diseases': 1,
 'Chemically-Induced Disorders': 2,
 'Congenital Hereditary and Neonatal Diseases and Abnormalities': 4,
 'Digestive System Diseases': 5,
 'Disorders of Environmental Origin': 22,
 'Endocrine System Diseases': 6,
 'Eye Diseases': 7,
 'Female Urogenital Diseases and Pregnancy Complications': 8,
 'Hemic and Lymphatic Diseases': 9,
 'Immune System Diseases': 10,
 'Male Urogenital Diseases': 11,
 'Musculoskeletal Diseases': 24,
 'Neoplasms': 12,
 'Nervous System Diseases': 13,
 'Nutritional and Metabolic Diseases': 14,
 'Occupational Diseases': 15,
 'Otorhinolaryngologic Diseases': 16,
 'Parasitic Diseases': 17,
 'Pathological Conditions and Signs and Symptoms': 18,
 'Respiratory Tract Diseases': 19,
 'Skin and Connective Tissue Diseases': 25,
 'Stomatognathic Diseases': 20,
 'Virus Diseases': 21,
 'Wounds and Injuries': 23}
test['categories'] = test['categories'].map(output_map)

In [ ]:
class DistillationBertTrain(Dataset):
    def __init__(self,type,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        if type == "train":
            self.len = len(train)
            self.abstract = list(train['abstract'])
            self.category = list(train['categories'])
        else:
            self.len = len(test)
            self.abstract = list(test['abstract'])
            self.category = list(test['categories'])
    def __getitem__(self, index):
        input = self.abstract[index]
        output = self.category[index]
  
        inputs = self.tokenizer.encode_plus(
            input,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(output, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
testing_set = DistillationBertTrain('test', tokenizer, MAX_LEN)
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
testing_loader = DataLoader(testing_set, **test_params)

## Distilled bert

In [ ]:
output_dir = '/content/gdrive/My Drive/lily/state_dict_model.pt'
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 26)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistillBERTClass()
model.load_state_dict(torch.load(output_dir))
model.to(device)

In [ ]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            nb_tr_examples+=targets.size(0)
            
    epoch_accu = (n_correct*100)/nb_tr_examples
   
    return epoch_accu

In [ ]:
accuracy = valid(model,testing_loader)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
accuracy

77.54557171791566

##  Accuracy of Distilled bert is 77.5%

## Bert

In [ ]:
output_dir = '/content/gdrive/My Drive/lily/bert_model/'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 26, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
output_dir = '/content/gdrive/My Drive/lily/bert_model/state_dict_model.pt'
model.load_state_dict(torch.load(output_dir))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum()
    return n_correct

In [ ]:
model.eval()
predictions = []
n_correct = 0; nb_tr_examples = 0
with torch.no_grad():
    for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits,axis=1).flatten()
        predictions.extend(logits)
        label_ids = targets.to('cpu').numpy()
        n_correct += calcuate_accu(logits, label_ids)
        nb_tr_examples+=targets.size(0)
        
accuracy = (n_correct*100)/nb_tr_examples
            


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
accuracy

82.62750874608727

In [ ]:
from sklearn.metrics import f1_score
print("macro:"+ str(f1_score(list(test['categories']), predictions, average='macro')*100))
print("micro: %.2f%%" % (f1_score(list(test['categories']), predictions, average='micro')*100))
print("weigted:"+ str(f1_score(list(test['categories']), predictions, average='weighted')*100))


macro:68.61098733186863
micro: 82.63%
weigted:82.20981210492029


## accuracy and f1-score of Bert model is 82% 

## BioBert

In [ ]:
MAX_LEN = 512
VALID_BATCH_SIZE = 16
tokenizer = BertTokenizer.from_pretrained('/content/gdrive/My Drive/lily/biobert/')

In [ ]:
model = BertForSequenceClassification.from_pretrained('/content/gdrive/My Drive/lily/biobert/new/')

In [ ]:
output_dir = '/content/gdrive/My Drive/lily/biobert/state_dict_model.pt'
model.load_state_dict(torch.load(output_dir))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
testing_set = DistillationBertTrain('test', tokenizer, MAX_LEN)
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model.eval()
predictions = []
n_correct = 0; nb_tr_examples = 0
with torch.no_grad():
    for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits,axis=1).flatten()
        predictions.extend(logits)
        label_ids = targets.to('cpu').numpy()
        n_correct += calcuate_accu(logits, label_ids)
        nb_tr_examples+=targets.size(0)
        
accuracy = (n_correct*100)/nb_tr_examples
            


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
accuracy

84.29386853249862

In [ ]:
from sklearn.metrics import f1_score
print("macro:"+ str(f1_score(list(test['categories']), predictions, average='macro')*100))
print("micro: %.2f%%" % (f1_score(list(test['categories']), predictions, average='micro')*100))
print("weigted:"+ str(f1_score(list(test['categories']), predictions, average='weighted')*100))


macro:73.05226530014356
micro: 84.29%
weigted:84.06152318829633


## Accuracy and f1-score of BioBert model is 84%

### Keras model

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('/content/gdrive/My Drive/lily/keras_model.h5')

In [ ]:
test = np.load('/content/gdrive/My Drive/lily/keras_test_data.npy')
labels = np.load('/content/gdrive/My Drive/lily/keras_test_label.npy')

In [ ]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
predictions = model.predict_classes(test, batch_size=64, verbose=0).ravel()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
print("Accuracy: %.2f%%" % (accuracy_score(test['categories'], predictions)*100))

Accuracy: 68.27%


##  Neural Network Language Model

In [ ]:
model = tf.keras.models.load_model('/content/gdrive/My Drive/lily/')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 26)                3354      
Total params: 124,679,066
Trainable params: 124,679,066
Non-trainable params: 0
__________________________________________

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', ' ', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
    return norm_docs

In [ ]:
test_content = pre_process_corpus(list(test['abstract']))

100%|██████████| 10862/10862 [00:01<00:00, 5737.67it/s]


In [ ]:
predictions = model.predict_classes(test_content, batch_size=64, verbose=0).ravel()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
print("Accuracy: %.2f%%" % (accuracy_score(test['categories'], predictions)*100))

Accuracy: 64.62%


## Inference

BioBert model work the best here, because its being finetuned on  biomedical citations and abstractions dataset.it leverages the represenation learnt from biomedical domain corpora.